# Part 1: Random Graph generators

In [1]:
import numpy as np 
import pandas as pd
import networkx as nx
import random
import math 

In [4]:
def random_graph(prob_resp=0.25,prob_nuevo=0.55, respondents_range=(20,30), connections_range=(9,18)):
    """
    prob_resp float    define la probabilidad de que al tomar un nodo este haya sido un respondiente de la encuesta
    prob_nuevo float   define la probabilidad de que un respondiente tenga conexiones con nodos nuevos

    Esta funcion genera graficas aleatorias basadas en el metodo con el que se crearon las networks de emprendimiento
    """
    G=nx.Graph()
    queue=["0"]
    max_index=0
    max_respondents=random.randint(respondents_range[0],respondents_range[1])             #Indicador que nos dice cuantos respondientes puede haber
    respondents=0
    while len(queue)!=0:
        if (random.random()<prob_resp and respondents<max_respondents) or queue[0]=="0":      #Crear conexiones para un respondiente
            respondents+=1
            n=random.randint(connections_range[0],connections_range[1])
            nx.set_node_attributes(G, {queue[0]:n},'responses')
            new_n=0
            old_n=0
            for i in range(n):                                              #Calcular cuantas conexiones son con nodos nuevos y cuantos con ya existentesz
                if random.random()<prob_nuevo:
                    new_n+=1
                else:
                    old_n+=1
            for i in range(new_n):
                new_node=str(i+max_index+1)
                queue.append(new_node)
                G.add_node(new_node)
                G.add_edge(queue[0],new_node)
            for i in range(old_n):
                node=str(random.randint(0,int(queue[0]))-1)
                G.add_edge(queue[0],node)
            max_index+=new_n
            queue.pop(0)
        else:                                                                #Crear conexiones para los no respondientes
            if random.random()<0.25:
                n=random.randint(1,6)
                for i in range(n):
                    node=str(random.randint(0,int(queue[0]))-1)
                    G.add_edge(queue[0],node)
            queue.pop(0)
    if len(G.nodes())<150 or len(G.nodes())>400:
        return random_graph(prob_resp,prob_nuevo)
    else:
        return G

""" Cambiar probabilidad de nuevas conexiones (ordenadas de menos a mas colaboracion)"""    
for i in range(100):
    nx.write_graphml(random_graph(prob_nuevo=0.55+(100-i)/400),'Random_Graphs_Nuevas_Conexiones/Random_Graph_Nuevas_Conexiones'+str(i)+'.graphml')
for i in range(100):
    nx.write_graphml(random_graph(prob_nuevo=0.55-i/400),'Random_Graphs_Nuevas_Conexiones/Random_Graph_Nuevas_Conexiones'+str(100+i)+'.graphml')

"""Cambiar probabilidad de que haya respuondientes del cuestionaro (no se si haya cambio en colaboracion posiblemente no)"""
for i in range(100):
    nx.write_graphml(random_graph(prob_resp=0.40-(100-i)/400),'Random_Graphs_Respondientes/Random_Graph_Respondientes'+str(i)+'.graphml')
for i in range(100):
    nx.write_graphml(random_graph(prob_nuevo=0.40+i/400),'Random_Graphs_Respondientes/Random_Graph_Respondientes'+str(100+i)+'.graphml')

"""Cambiar rango de numero de respondientes por grafica (de menos a mas colaboracion)"""
for i in range(100):
    nx.write_graphml(random_graph(respondents_range=(int(20-(100-i)/10),int(30-(100-i)/10))),'Random_Graphs_Rango_Respondientes/Random_Graph_Rango_Respondientes'+str(i)+'.graphml')
for i in range(100):
    nx.write_graphml(random_graph(respondents_range=(int(20+i/10),int(30+i/10))),'Random_Graphs_Rango_Respondientes/Random_Graph_Rango_Respondientes'+str(100+i)+'.graphml')

"""Cambiar rango de numero de respuestas por respondiente (de menos a mas colaboracion)"""
for i in range(100):
    nx.write_graphml(random_graph(respondents_range=(int(15-(100-i)/10),int(24-(100-i)/10))),'Random_Graphs_Numero_Respuestas/Random_Graph_Numero_Respuestas'+str(i)+'.graphml')
for i in range(100):
    nx.write_graphml(random_graph(respondents_range=(int(15+i/10),int(24+i/10))),'Random_Graphs_Numero_Respuestas/Random_Graph_Numero_Respuestas'+str(100+i)+'.graphml')

In [130]:
from numpy.random import choice

def are_adjacent(u,v,G):
    if v not in G.nodes():
        return False
    elif v in G.neighbors(u):
        return True
    else:
        return False

def random_graph_2(n_evals, prob_out, prob_new):
    '''
    Generator of a random graph, given that n
    Given a specific number of respondents of the questionnaire, and that each of them could have provided 25 responses maximum
    
    Input:
     - n_evals  : number of evaluators responding questionnaire
     - prob_out : probability that a mentionned collaboration is outside of the network of evaluators
    
    '''
    
    G=nx.DiGraph()
    setedges = set()
    
    inside = dict()
    outside = dict()
    
    last_index = n_evals-1
    
    for i in range(n_evals):              #add all evaluators to the graph
        G.add_node(i)
        inside[i] = 0
        
    # evaluator |--> no. of collaborations,   in (0,25)~ normal distribution
    sample_num_evaluations = np.random.normal(loc=12.3, scale=4, size=n_evals)
    list_degrees_evaluators = [int(x) for x in sample_num_evaluations]
    
    for i in inside.keys():
        for j in range(list_degrees_evaluators[i]):
            
            if random.random()<prob_out:           #edge added outside evaluators' list
                if outside == dict():                              # if list outside evaluators is new
                    last_index+=1
                    to = last_index
                    G.add_node(to)
                    G.add_edge(i,to)
                    setedges.add((i,to))
                    outside[to] = 1
                    inside[i] += 1
                else:
                    if random.random()<prob_new:               # if edge goes to a new vertex
                        last_index+=1       
                        to=last_index
                        G.add_node(to)
                        G.add_edge(i,to)
                        setedges.add((i,to))
                        outside[to] = 1
                        inside[i] +=1
                        
                    else:
                        vertices, degrees = [], []
                        for vertex, degree in outside.items():
                            if vertex!=i:
                                vertices.append(vertex)
                                degrees.append(degree+1)
                        s = sum(degrees)
                        w = [x/s for x in degrees]
                        to = choice(vertices, size=1, p=w)[0]
                        if are_adjacent(i,to,G):
                            to = choice(vertices, size=1, p=w)[0]
                        G.add_node(to)
                        G.add_edge(i,to)
                        setedges.add((i,to))
                        outside[to]+=1
                        inside[i]+=1
                        
            else:
                vertices, degrees = [], []
                for vertex, degree in inside.items():
                    if vertex != i:
                        vertices.append(vertex)
                        degrees.append(degree+1)
                s = sum(degrees)
                w = [x/s for x in degrees]
                to = choice(vertices, size=1, p=w)[0]
                if are_adjacent(i,to,G):
                    to = choice(vertices, size=1, p=w)[0]
                G.add_node(to)
                G.add_edge(i,to)
                setedges.add((i,to))
                inside[to]+=1
                inside[i]+=1
    return G         


list_cities = ['Aguascalientes', 'Buenos Aires', 'Ciudad de México', 'Guadalajara', 'Hidalgo',
                  'Madrid', 'Montevideo', 'Oaxaca', 'Sao Paulo', 'Santiago de Chile']
num_evaluators = {'Aguascalientes':19, 'Buenos Aires':31, 'Ciudad de México':36, 'Guadalajara':32, 'Hidalgo':19,
                  'Madrid':37, 'Montevideo':48, 'Oaxaca':36, 'Sao Paulo':28, 'Santiago de Chile':25}
for city in list_cities:
    G= random_graph_2(num_evaluators[city],0.2, 0.5)
    nx.write_graphml(G,'Random_Graphs_Second_Type_Corrected/Random_Graph_'+city+'.graphml')


In [131]:
muestra = pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - Muestra.csv')             


ags_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - AGS.csv')             
caba_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - CABA.csv')             
cdmx_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - CDMX.csv')             
gdl_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - GDL.csv')             
hgo_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - HGO.csv')             
mad_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - MAD.csv')             
mtv_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - MVD.csv')             
oax_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - OAX.csv')             
sao_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - SAO.csv')             
scl_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - SCL.csv')  

ags_info.rename(columns={'Unnamed: 1':'Colabs'}, inplace=True)
gdl_info.rename(columns={'Unnamed: 1':'Colabs'}, inplace=True)
hgo_info.rename(columns={'Unnamed: 1':'Colabs'}, inplace=True)
oax_info.rename(columns={'Unnamed: 1':'Colabs'}, inplace=True)


list_cities = ['AGS', 'CABA', 'CDMX', 'GDL', 'Pachuca',
                  'Madrid', 'Montevideo', 'Oaxaca', 'Sao Paulo', 'Santiago']
cities_info = {'AGS': ags_info, 
              'CABA' : caba_info,
              'CDMX' : cdmx_info,
              'GDL' : gdl_info,
              'Pachuca' : hgo_info,
              'Madrid' : mad_info,
              'Montevideo' : mtv_info,
              'Oaxaca' : oax_info,
              'Sao Paulo' : sao_info,
              'Santiago' : scl_info}

muestra

,Unnamed: 0,Madrid,Madrid SG,Madrid Typeform,CDMX,Santiago,CABA,Sao Paulo,Montevideo,Oaxaca,GDL,Pachuca,AGS
0,Muestra,38,16,24,51,30,36,34,59,36,32,21,19
1,Nodes,239,120,156,299,195,228,216,198,149,187,125,96
2,Edges,453,166,205,547,385,500,364,767,326,474,254,233
3,Promedio colabs. por participante,12.13,10.13,8.63,12.33,13.04,13.48,10.38,13.4,6.39,10.81,9.52,8.21
4,% Muestra,15.90%,13.33%,15.38%,17.06%,15.38%,15.79%,15.74%,29.80%,24.16%,17.11%,16.80%,19.79%


In [134]:
citygraphs = dict()
for city in list_cities:
    citygraphs[city] = nx.read_graphml('GraphMLs/'+city+' original graph.graphml')

#citygraphs

{'AGS': <networkx.classes.digraph.DiGraph at 0xa22a59350>,
 'CABA': <networkx.classes.digraph.DiGraph at 0xa25710510>,
 'CDMX': <networkx.classes.digraph.DiGraph at 0xa25710f90>,
 'GDL': <networkx.classes.digraph.DiGraph at 0xa25710dd0>,
 'Pachuca': <networkx.classes.digraph.DiGraph at 0xa25710690>,
 'Madrid': <networkx.classes.digraph.DiGraph at 0xa22c93110>,
 'Montevideo': <networkx.classes.digraph.DiGraph at 0xa252f7e50>,
 'Oaxaca': <networkx.classes.digraph.DiGraph at 0xa2548fe90>,
 'Sao Paulo': <networkx.classes.digraph.DiGraph at 0xa25c0d350>,
 'Santiago': <networkx.classes.digraph.DiGraph at 0xa26f05dd0>}

In [86]:
'''Third Random Graph Generator'''

def eliminate_small_components(G):
    '''
    We make sure there are no disconnected components. 
    '''
    to_remove = []
    
    for x in G.nodes():
        if len(list(nx.node_connected_component(nx.to_undirected(G),x)))< 20 :
            to_remove.append(x)
    for x in to_remove:
        G.remove_node(x)
    
    if nx.number_connected_components(nx.to_undirected(G))>1:
        print('****')    
    return G



def random_graph_3(infos, city, prob_out, prob_new):
    '''
    With this function, we are simulating the responses we obtained from each of the surveyed ecosystems/cities. 
    The information we have is, for each evaluator, how many 
    
    Generator of a random graph, given the number of respondents of the questionnaire,
    the number of evaluators of collaborations, and the number of collaborations reported by each of them.
    
    Input:
     - infos : dictionary mapping from city to num_collab information
     - city : name of city to be simulated
     - prob_out : probability that a mentionned collaboration is outside of the network of evaluators
     - prob_new : probability that a mentionned collaboration is a newly mentionned org
     
    '''
    
    G=nx.DiGraph()
    setedges = set()
    
    
    #degrees of nodes inside and outside respondent list
    inside = dict()
    outside = dict()
    
    
    #determine number of organizations that answered and number of evaluators: 
    n_responded = int(muestra[city][0])
    n_evals = len(list(infos[city]['Colabs']))
    last_index = n_responded
    
    
    #number of total nodes, and number of edges:                  #still don't know how to use
    num_nodes = int(muestra[city][1])
    num_edges = int(muestra[city][2])
    
    for i in range(last_index):              #add all evaluators to the graph
        G.add_node(i)
        if i in range(n_evals):
            inside[i] = 0
        else:
            outside[i] = 0
        
    
    for i in inside.keys():
        
        deg_i = infos[city]['Colabs'][i]         # the degree of that node is in the info retrieved
        for j in range(deg_i):
            
            if random.random() < prob_out:           #edge added outside evaluators' list
                if outside == dict():                              # if list outside evaluators is new
                    #last_index+=1
                    to = last_index
                    G.add_node(to)
                    G.add_edge(i,to)
                    setedges.add((i,to))
                    outside[to] = 1
                    inside[i] += 1
                else:
                    if random.random()<prob_new:               # if edge goes to a new vertex
                        last_index+=1       
                        to=last_index
                        G.add_node(to)
                        G.add_edge(i,to)
                        setedges.add((i,to))
                        outside[to] = 1
                        inside[i] +=1
                        
                    else:
                        vertices, degrees = [], []
                        for vertex, degree in outside.items():
                            if vertex!=i:
                                vertices.append(vertex)
                                degrees.append(degree+1)
                        s = sum(degrees)
                        w = [x/s for x in degrees]
                        to = choice(vertices, size=1, p=w)[0]
                        if are_adjacent(i,to,G):
                            to = choice(vertices, size=1, p=w)[0]
                        G.add_edge(i,to)
                        setedges.add((i,to))
                        outside[to]+=1
                        inside[i]+=1
                        
            else:
                vertices, degrees = [], []
                for vertex, degree in inside.items():
                    if vertex != i:
                        vertices.append(vertex)
                        degrees.append(degree+1)
                s = sum(degrees)
                w = [x/s for x in degrees]
                to = choice(vertices, size=1, p=w)[0]
                if are_adjacent(i,to,G):
                    to = choice(vertices, size=1, p=w)[0]
                G.add_edge(i,to)
                setedges.add((i,to))
                inside[to]+=1
                inside[i]+=1
                
    # now we have the degree of each evaluator covered, 
    # but also some of the orgs responded with
    # 2 key orgs in their development and establishment
    
    for i in range(n_responded):
        for j in range(2):

            if random.random()<prob_out:           #edge added outside evaluators' list
                if random.random()<prob_new:               # if edge goes to a new vertex
                    last_index+=1       
                    to=last_index
                    G.add_node(to)
                    G.add_edge(i,to)
                    setedges.add((i,to))
                    outside[to] = 1
                    if i in outside.keys():
                        outside[i]+=1  
                    elif i in inside.keys():
                        inside[i]+=1 
                        
                else:
                    vertices, degrees = [], []
                    for vertex, degree in outside.items():
                        if vertex!=i:
                            vertices.append(vertex)
                            degrees.append(degree+1)
                    s = sum(degrees)
                    w = [x/s for x in degrees]
                    to = choice(vertices, size=1, p=w)[0]
                    if are_adjacent(i,to,G):
                        to = choice(vertices, size=1, p=w)[0]
                    G.add_edge(i,to)
                    setedges.add((i,to))
                    outside[to]+=1
                    if i in outside.keys():
                        outside[i]+=1  
                    elif i in inside.keys():
                        inside[i]+=1 
                        
            else:
                vertices, degrees = [], []
                for vertex, degree in inside.items():
                    if vertex != i:
                        vertices.append(vertex)
                        degrees.append(degree+1)
                s = sum(degrees)
                w = [x/s for x in degrees]
                to = choice(vertices, size=1, p=w)[0]
                if are_adjacent(i,to,G):
                    to = choice(vertices, size=1, p=w)[0]
                G.add_edge(i,to)
                setedges.add((i,to))
                inside[to]+=1
                if i in outside.keys():
                    outside[i]+=1  
                elif i in inside.keys():
                    inside[i]+=1  
    
    F=eliminate_small_components(G)
    return F
                
            
for city in list_cities:
    G = random_graph_3(cities_info,city,0.8, 0.6)
    nx.write_graphml(G,'Random_Graphs_Third_Type_Corrected/Random_Graph_'+city+'.graphml')

# Part 2: Helper functions

We work with the helper functions for the four different types of average, and for the different measures we can take from an ecosystem.  

In [87]:
#-----------------------
# Averages
#-----------------------

def quadratic(L):
    return ( sum([x**2 for x in L])/len(L) )**0.5
    
def arithmetic(L):
    return sum(L)/len(L)

def geometric(L):
    prod = 1
    for x in L:
        prod *=x
    return prod**(1/len(L))

def harmonic(L):
    n = len(L)
    sum_reciprocals = sum([1/x for x in L])
    return n/sum_reciprocals


mean_map = {'quadratic': quadratic, 
           'arithmetic': arithmetic, 
           'geometric': geometric, 
           'harmonic': harmonic}

In [93]:
#-----------------------
# Graph metrics!
#-----------------------

def degree(G):
    return np.mean([G.degree(x) for x in G.nodes()])

def eccentricity(U):
    return np.mean([nx.eccentricity(U,x) for x in U.nodes()])

def clustering(U):
    return np.mean([nx.clustering(U,x) for x in U.nodes()]) 

def diameter(U):
    return nx.diameter(U)

def radius(U):
    return nx.radius(U)

def avg_shortest_path_length(U):
    return nx.average_shortest_path_length(U)

def transitivity(U):
    return nx.transitivity(U)

def global_efficiency(U):
    return nx.global_efficiency(U)

def small_worldness(U):
    return nx.algorithms.smallworld.sigma(U,niter=1,nrand=2)

def rich_club_coeffs(G):
    t_ok = True
    t = 0
    d = dict()
    while t_ok:
        nodes_large_degree=[]
        for x in G.nodes():
            if G.degree(x)>t:
                nodes_large_degree.append(x)
        core = G.subgraph(nodes_large_degree)
        edges_core = len(core.edges())
        nodes_core = len(core.nodes())
        if nodes_core<=1:
            t_ok = False
            break
        d[t] = (2*edges_core)/(nodes_core*(nodes_core-1))
        t += 1 
    return d  

def max_rich_club(G):
    rich_club=rich_club_coeffs(G)
    max_i=0
    
    for i in range(len(rich_club)):
        if rich_club[i]>rich_club[max_i]:
            max_i=i
    return rich_club[max_i]

def core_ratio(G):
    return len(nx.k_core(G,k=2).nodes())/len(G.nodes())
        
def central_point_dominance(G):
    betwennesses = nx.betweenness_centrality(G)
    b_max = max(betwennesses.values())
    N = len(betwennesses.keys())
    count = 0
    for i, b_i in betwennesses.items():
        count += ( b_max - b_i )/(N-1)
    return count


def spectral_radius(G):
    L = nx.normalized_laplacian_matrix(G)
    e = np.linalg.eigvals(L.A)
    e_abs = [abs(x) for x in e]
    return max(e_abs)

def modularity(G):
    return nx.algorithms.community.quality.performance(G,nx.algorithms.community.modularity_max.greedy_modularity_communities(G))



        
metric_function_map = {'Degree': degree,
            'Excentricidad': eccentricity,
            'Clustering': clustering,
            'Diámetro': diameter,
            'Radio':radius,
            'Camino más corto promedio':avg_shortest_path_length,
            'Transitividad':transitivity,
            'Eficiencia Global':global_efficiency,
            'Rich Club Coefficient':max_rich_club,
            'Core Ratio':core_ratio,
            'Central Point Dominance':central_point_dominance,
            'Spectral radius':spectral_radius,
            'Modularidad':modularity} 


def measures(G):
    D = dict()
    for metric, function in metric_function_map.items():
        D[metric] = function(nx.to_undirected(G))
    return D

def embed(D1, D2):
    for key, value in D1.items():
        D2[key].append(value)
    return D2


# Part 3: Colaborativity functions

In [89]:
def colaborativity(G, set_comunication, set_preparation, set_resilience, meantype):
    '''
    Computes the colaborativity of a network G based on sets which determine which metrics are to be computed
    
    - set_communication : set containing the names of the metrics that will measure communication pilar
    - set_preparation : contains metrics that will measure preparation for future collabs pilar
    - set_resilience : contins the names of the metrics that will measure resilience pilar
    '''
    comm, prep, resi = [], [], []
    U = nx.to_undirected(G)
    
    for metric in set_communication:
        comm.append( metric_function_map[metric](U) )
    comm = arithmetic(comm)
    
    for metric in set_preparation:
        prep.append( metric_function_map[metric](U) )
    prep = arithmetic(prep)

    
    for metric in set_resilience:
        resi.append( metric_function_map[metric](U) )
    resi = arithmetic(resi)
        
    #now we compute the mean
    
    return mean_map[meantype]([comm, prep, resi])


G = random_graph_3(cities_info,'GDL',0.8, 0.6)
set_communication = {'Eficiencia Global'}
set_preparation = {'Rich Club Coefficient'}
set_resilience = {'Transitividad'}
meantype = 'arithmetic'

print(colaborativity(G, set_communication, set_preparation, set_resilience, meantype))

0.2381281213064996


In [90]:
def colaborativity_from_dataframe(G, index, df, set_comunication, set_preparation, set_resilience, meantype):
    '''
    Computes the colaborativity of a network G based on sets which determine which metrics are to be computed
    
    - set_communication : set containing the names of the metrics that will measure communication pilar
    - set_preparation : contains metrics that will measure preparation for future collabs pilar
    - set_resilience : contins the names of the metrics that will measure resilience pilar
    '''
    comm, prep, resi = [], [], []
    U = nx.to_undirected(G)
    
    for metric in set_communication:
        comm.append( list(df[metric])[index] )
    comm = arithmetic(comm)
    
    for metric in set_preparation:
        prep.append( list(df[metric])[index] )
    prep = arithmetic(prep)

    
    for metric in set_resilience:
        resi.append( list(df[metric])[index] )
    resi = arithmetic(resi)
        
    #now we compute the mean
    
    return mean_map[meantype]([comm, prep, resi])

In [137]:
def colaborativity_original(datadict, G, real_city = None):
    '''
    Original Collaborativity Formula during summer 2020 for project GED
    Computes the colaborativity of a network G based on its measures, as shown below. 
    
    Input:
    - datadict : dictionary mapping Metric to Metric(G), for some real or synthetic graph G
    Returns:
    - Value between (technically) 0 and 3.5, describing the collaborativity of a graph, or network
    
    '''
    #Avg_colabs = datadict['Degree']
    if real_city:
        Avg_colabs = np.mean(list(cities_info[real_city]['Colabs']))
    else:
        Avg_colabs = np.mean([minus_two(G,x) for x in G.nodes()])

    Clust = datadict['Clustering']
    Mod = datadict['Modularidad']
    return (1/2) * Avg_colabs * (Clust + (np.log(Mod**2)*-1))

def minus_two(G,x):
    if G.out_degree(x) < 2:
        return 0
    else:
        return G.out_degree(x)-2


def colaborativity_formula_1(datadict):
    '''
    First Proposed Collaborativity Formula in GED summer 2020 participation in the project
    Computes the colaborativity of a network G based on its measures, as shown below. 
    
    Input:
    - datadict : dictionary mapping Metric to Metric(G), for some real or synthetic graph G
    Returns:
    - Value between (technically) 0 and 3.5, describing the collaborativity of a graph, or network
    
    '''
    Efi = datadict['Eficiencia Global']
    Tran = datadict['Transitividad']
    Mod = datadict['Modularidad']
    Core = datadict['Core Ratio']
    return Efi + Tran + 1 - (Mod + Core)/2 




def colaborativity_formula_2(datadict):
    '''
    Second Proposed Collaborativity Formula in GED summer 2020 participation in the project
    Computes the colaborativity of a network G based on its measures, as shown below. 
    
    Input:
    - datadict : dictionary mapping Metric to Metric(G), for some real or synthetic graph G
    Returns:
    - Value between (technically) 0 and 3, describing the collaborativity of a graph, or network
    
    '''
    Efi = datadict['Eficiencia Global']
    Tran = datadict['Transitividad']
    Exc = datadict['Excentricidad']
    Mod = datadict['Modularidad']
    return Efi + Tran + (1/Exc) - Mod


def colaborativity_formula_3(datadict):
    '''
    Second Proposed Collaborativity Formula in GED summer 2020 participation in the project
    Computes the colaborativity of a network G based on its measures, as shown below. 
    
    Input:
    - datadict : dictionary mapping Metric to Metric(G), for some real or synthetic graph G
    Returns:
    - Value between (technically) 0 and 3, describing the collaborativity of a graph, or network
    
    '''
    Efi = datadict['Eficiencia Global']
    Clus = datadict['Clustering']
    Mod = datadict['Modularidad']
    #print(Mod)

    return 1/3 * (  Efi + Clus + (1/2)* (1 + math.cos(math.pi*Mod))  )    


# Part 4: Monte Carlo Simulations

In [138]:
#----------------------------
# Monte Carlo Cooler Version
#----------------------------

tidydata = pd.read_csv('Olga/Tidy_DataFrame.csv')

def monte_carlo_2(infos, city, prob_out, prob_new, num_trials):
    
    data = {key : [] for key in metric_function_map.keys()}
    data['Name'] = []
    data['Colaboratividad Original'] = []
    data['Colaboratividad 1'] = []
    data['Colaboratividad 2'] = []
    data['Colaboratividad 3'] = []

    
    #Samples
    for i in range(num_trials):
        print(i)
        G = random_graph_3(infos,city,prob_out, prob_new)
        M = measures(G)
        
        data = embed(M,data)
        data['Name'].append('G'+str(i))
        data['Colaboratividad Original'].append(colaborativity_original(M, G))
        data['Colaboratividad 1'].append(colaborativity_formula_1(M))
        data['Colaboratividad 2'].append(colaborativity_formula_2(M))
        data['Colaboratividad 3'].append(colaborativity_formula_3(M))

        
    #Means
    for key in data.keys():
        if key != 'Name':
            data[key].append(np.mean( data[key] ))
    data['Name'].append('Mean')
    
    
    #Real Data
    citydata = pd.DataFrame(tidydata.loc[tidydata['Ciudad'] == city])
    citymeasures = {metric: list(tidydata.loc[tidydata['Ciudad'] == city][metric])[0] for metric in metric_function_map.keys()}
    
    data = embed(citymeasures, data)
    data['Name'].append(city+' real')
    G = citygraphs[city]
    #print(list(cities_info[city]['Colabs']))
    data['Colaboratividad Original'].append(colaborativity_original(M, G, city))
    data['Colaboratividad 1'].append(colaborativity_formula_1(citymeasures))
    data['Colaboratividad 2'].append(colaborativity_formula_2(citymeasures))
    data['Colaboratividad 3'].append(colaborativity_formula_3(citymeasures)) 

    
    #Incorporating the dataframe! 
    #'avg strength', 'weight', 'Weighted Degree','Small Worldness',
    df = pd.DataFrame(data, columns = ['Name','Degree', 'Excentricidad', 
                                       'Clustering', 'Diámetro', 'Radio', 'Camino más corto promedio', 
                                       'Transitividad', 'Eficiencia Global',
                                       'Rich Club Coefficient', 'Core Ratio', 'Central Point Dominance',
                                       'Spectral radius', 'Modularidad', 'Colaboratividad Original', 
                                       'Colaboratividad 1', 'Colaboratividad 2', 'Colaboratividad 3'])
    
    return df



#let's try to test this function
monte_carlo_2(cities_info, 'Montevideo', 0.8, 0.6, 10)




0
1
2
3
4
5
6
7
8
9


,Name,Degree,Excentricidad,Clustering,Diámetro,Radio,Camino más corto promedio,Transitividad,Eficiencia Global,Rich Club Coefficient,Core Ratio,Central Point Dominance,Spectral radius,Modularidad,Colaboratividad Original,Colaboratividad 1,Colaboratividad 2,Colaboratividad 3
0,G0,3.647343,5.188406,0.045833,6.0,4.0,3.583208,0.035113,0.300533,0.250000,0.437198,0.097208,1.834811,0.934671,0.140086,0.649712,-0.406288,0.118953
1,G1,3.612440,5.377990,0.068883,6.0,4.0,3.651613,0.065224,0.295703,1.000000,0.413876,0.125191,1.851621,0.915436,0.187132,0.696272,-0.368565,0.127376
2,G2,3.518692,5.271028,0.046142,6.0,4.0,3.673094,0.040414,0.293561,1.000000,0.392523,0.121982,1.855131,0.932292,0.138246,0.671568,-0.408600,0.116991
3,G3,3.852417,5.137405,0.057831,6.0,4.0,3.527496,0.054711,0.306056,0.666667,0.450382,0.105066,1.825609,0.928610,0.167443,0.671271,-0.373193,0.125470
4,G4,3.652174,4.987923,0.056927,6.0,4.0,3.608462,0.033971,0.298727,0.666667,0.442029,0.090195,1.841484,0.925021,0.163972,0.649173,-0.391838,0.123154
5,G5,3.498840,5.317865,0.037776,6.0,4.0,3.674138,0.044530,0.293197,0.500000,0.387471,0.117853,1.868290,0.924017,0.144939,0.681983,-0.398245,0.115050
6,G6,3.508197,5.913349,0.063246,7.0,4.0,3.719640,0.043577,0.291074,1.000000,0.414520,0.120814,1.880310,0.922992,0.165674,0.665895,-0.419232,0.122960
7,G7,3.432494,5.418764,0.058092,7.0,4.0,3.670155,0.038619,0.293094,0.333333,0.393593,0.104831,1.841559,0.926962,0.152652,0.671436,-0.410705,0.121430
8,G8,3.581948,5.902613,0.041675,7.0,4.0,3.656114,0.044157,0.295379,0.666667,0.418052,0.090921,1.850282,0.930981,0.139957,0.665020,-0.422027,0.116254
9,G9,3.586698,5.154394,0.067139,7.0,4.0,3.611073,0.062579,0.298209,0.666667,0.429929,0.114605,1.859935,0.920111,0.176772,0.685768,-0.365314,0.127004


In [139]:
for city in cities_info.keys():
    print(city)
    df = monte_carlo_2(cities_info, city, 0.8, 0.6, 50)
    df.to_csv('Data_Cities_wrt_Random/Trials_'+city+'.csv',index=False)


AGS
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
CABA
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
CDMX
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
GDL
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Pachuca
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Madrid
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Montevideo
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41


In [140]:
list(tidydata.loc[tidydata['Ciudad'] == city]['Clustering'])

[0.1340367487179487]

In [141]:
dataframe = pd.read_csv('Data_Cities_wrt_Random/Trials_Oaxaca.csv')
dataframe

,Name,Degree,Excentricidad,Clustering,Diámetro,Radio,Camino más corto promedio,Transitividad,Eficiencia Global,Rich Club Coefficient,Core Ratio,Central Point Dominance,Spectral radius,Modularidad,Colaboratividad Original,Colaboratividad 1,Colaboratividad 2,Colaboratividad 3
0,G0,3.200000,5.972973,0.045333,8.00,4.00,3.808637,0.064276,0.295258,1.000000,0.470270,0.186360,1.912412,0.924559,0.122419,0.662118,-0.397605,0.118189
1,G1,3.224044,5.994536,0.031927,7.00,4.00,3.736024,0.068087,0.299566,0.666667,0.459016,0.252377,1.899466,0.903201,0.143517,0.686545,-0.368729,0.118145
2,G2,3.461988,5.251462,0.049009,6.00,4.00,3.541658,0.063856,0.316410,0.400000,0.438596,0.323008,1.900281,0.908359,0.158005,0.706788,-0.337670,0.128666
3,G3,3.397727,5.255682,0.038062,6.00,4.00,3.577922,0.059689,0.311718,1.000000,0.471591,0.245420,1.896103,0.910844,0.144989,0.680189,-0.349168,0.123088
4,G4,3.325843,5.134831,0.043916,6.00,4.00,3.545166,0.055505,0.312344,1.000000,0.488764,0.287288,1.876723,0.905859,0.152055,0.670537,-0.343262,0.125989
5,G5,3.256831,5.672131,0.046561,7.00,4.00,3.825797,0.053774,0.294227,1.000000,0.453552,0.210832,1.911199,0.897856,0.161813,0.672296,-0.373555,0.122104
6,G6,3.248619,6.685083,0.036524,9.00,5.00,3.761203,0.053715,0.301743,1.000000,0.430939,0.208620,1.956966,0.901596,0.149452,0.689191,-0.396551,0.120657
7,G7,3.104167,5.598958,0.040513,7.00,4.00,3.803501,0.043785,0.294227,0.333333,0.421875,0.185497,1.904209,0.921302,0.120326,0.666424,-0.404685,0.116648
8,G8,3.310734,5.587571,0.066496,7.00,4.00,3.705637,0.062500,0.302483,0.666667,0.497175,0.259515,1.919549,0.911916,0.156642,0.660438,-0.367964,0.129334
9,G9,3.360465,5.534884,0.032415,7.00,4.00,3.676391,0.052343,0.305162,1.000000,0.482558,0.180747,1.886551,0.911873,0.138100,0.660290,-0.373695,0.118873
